# 6.7 门控循环单元（GRU）
## 6.7.2 读取数据集

In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 数据资料
# 时间序列          字符:索引        字符        去重字符数 
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()
print(torch.__version__, device)

2.3.0+cpu cpu


C:\Users\fanxiaolong\anaconda3\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
C:\Users\fanxiaolong\anaconda3\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


## 6.7.3 从零开始实现
### 6.7.3.1 初始化模型参数

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cpu


### 6.7.3.2 定义模型

In [3]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [4]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

### 6.7.3.3 训练模型并创作歌词

In [5]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e-2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [6]:
print('要求代数:',num_epochs)
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

要求代数: 160
不随机迭代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代


## 6.7.4 简洁实现

In [7]:
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总代数):  8
第0代
第1代
第2代
第3代
第4代
第5代
第6代
第7代
epoch_size(总